In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf

In [ ]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='rmsprop',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

In [ ]:
train_dir = 'dataset/train_images'
val_dir = 'dataset/val_images'
test_dir = 'dataset/test_images'
train_dir2 = 'train'

#datagen = ImageDataGenerator(1./255)
train_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir2,
    validation_split=0.2,
    batch_size=64,
    image_size=(256,256),
    seed=123,
    color_mode='rgb',
    subset = 'training'
)
val_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir2,
    validation_split=0.2,
    batch_size=64,
    image_size=(256,256),
    seed=123,
    color_mode='rgb',
    subset = 'validation'
)
print(train_generator.class_names)

In [ ]:
history = model.fit(train_generator,
                    validation_data = val_generator,
                    epochs=5)

In [ ]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [ ]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

"""model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])"""

In [ ]:
history = model.fit(train_generator,
                    validation_data = val_generator,
                    epochs=40)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image

img = image.load_img('dataset/test_images/fake/df2.jpg', target_size=(256,256))
img_1 = Image.open('dataset/test_images/fake/df2.jpg')
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

img2 = image.load_img('dataset/test_images/real/real00240.jpg', target_size=(256,256))
img_2 = Image.open('dataset/test_images/real/real00240.jpg')
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2, axis = 0)

classes = ['fake','real']

#print(classes[np.argmax(model.predict(img))])
#print(classes[np.argmax(model.predict(img2))])

plt.imshow(img_1)
plt.title(classes[np.argmax(model.predict(img))])

In [ ]:
plt.imshow(img_2)
plt.title(classes[np.argmax(model.predict(img2))])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(40)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#model.save('model/inception/inceptionv3_3')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('model/inception/inceptionv3_2')

In [ ]:
score = model.evaluate(val_generator)
print(score[1])